In [ ]:
%matplotlib inline
import datetime
import matplotlib.pyplot as plt
import glob
import os
from obspy import read, UTCDateTime, read_inventory
from obspy.signal import PPSD
import warnings
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib.dates import DateFormatter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
plt.style.use("ggplot")

from msnoise.api import connect, get_results, build_movstack_datelist, get_params, get_t_axis, get_results_all

In [ ]:
! msnoise db execute "insert into filters (ref, low, mwcs_low, high, mwcs_high, mwcs_wlen, mwcs_step, used) values (1, 0.1, 0.1, 1.0, 1.0, 12.0, 4.0, 1)"
! msnoise db execute "insert into filters (ref, low, mwcs_low, high, mwcs_high, mwcs_wlen, mwcs_step, used) values (2, 1, 1, 2, 2, 12.0, 4.0, 1)"
! msnoise db execute "insert into filters (ref, low, mwcs_low, high, mwcs_high, mwcs_wlen, mwcs_step, used) values (3, 2, 2, 4, 4, 12.0, 4.0, 1)"
! msnoise db execute "insert into filters (ref, low, mwcs_low, high, mwcs_high, mwcs_wlen, mwcs_step, used) values (4, 4, 4, 8, 8, 2.0, 1.0, 1)"

In [ ]:
! msnoise config set resampling_method=Decimate

In [ ]:
# ! msnoise -t 3 -d 5 -v cc compute_cc

In [ ]:
# connect to the database
db = connect()

# Obtain a list of dates between ``start_date`` and ``enddate``
start, end, datelist = build_movstack_datelist(db)

# Get the list of parameters from the DB:
params = get_params(db)

# Get the time axis for plotting the CCF:
taxis = get_t_axis(db)

filter_id = 1
mov_stack = 1

# Get the results for two station, filter id=1, ZZ component, mov_stack=1 and the results as a 2D array:
n, ccfs = get_results(db, "BE.MEM.--", "GR.AHRW.--", filter_id, "ZZ", datelist,
                      mov_stack, format="matrix", params=params)

# Convert to a pandas DataFrame object for convenience, and drop empty rows:
df = pd.DataFrame(ccfs, index=pd.DatetimeIndex(datelist), columns=taxis)
df = df.dropna()

# Define the 99% percentile of the data, for visualisation purposes:
clim = df.mean(axis="index").quantile(0.999)

fig, ax = plt.subplots(figsize=(12,8))
plt.pcolormesh(df.columns, df.index.to_pydatetime(), df.values,
               vmin=-clim, vmax=clim, rasterized=True, cmap="RdBu")
plt.colorbar()
plt.title("Interferogram")
plt.xlabel("Lag Time (s)")
plt.ylim(df.index[0],df.index[-1])
plt.xlim(df.columns[0], df.columns[-1])
plt.subplots_adjust(left=0.15)

Compute the reference stack and plot vs distance:

In [ ]:
! msnoise config sync
! msnoise cc stack -r

In [ ]:
!msnoise cc plot distance -f 1

reset the STACK jobs and run the mov_stack

In [ ]:
! msnoise reset STACK
! msnoise config set mov_stack=1,2,5
! msnoise cc stack -m


compute the mwcs & dtt

In [ ]:
! msnoise cc dvv compute_mwcs
! msnoise cc dvv compute_dtt


plot the final results

examples:

- msnoise cc dvv plot dvv -f 1
- msnoise cc dvv plot dvv -f 2 -m 5

In [ ]:
def get_dvv(session, filterid, components, dates,
                mov_stack=1, center="median") :
    dttname="M"
    pairs=[]

    current = dates[0]
    end = dates[-1]

    alldf = []
    while current <= end:
        for comp in components:
            day = os.path.join('DTT', "%02i" % filterid, "%03i_DAYS" %
                               mov_stack, components, '%s.txt' % current)
            if os.path.isfile(day):
                df = pd.read_csv(day, header=0, index_col=0,
                                 parse_dates=True)
                alldf.append(df)
        current += datetime.timedelta(days=1)
    if len(alldf) == 0:
        print("No Data for %s m%i f%i" % (components, mov_stack, filterid))

    alldf = pd.concat(alldf)
    print(mov_stack, alldf.head())
    if 'alldf' in locals():
        errname = "E" + dttname
        alldf.to_csv("tt.csv")
        alldf[dttname] *= -100
        alldf[errname] *= -100

        allbut = alldf[alldf['Pairs'] != 'ALL'].copy()

        for pair in pairs:
            print(pair)
            pair1 = alldf[alldf['Pairs'] == pair].copy()
            print(pair1.head())

            pair1.to_csv('%s-m%i-f%i.csv'%(pair, mov_stack, filterid))

        if center == "median":
            tmp3 = allbut[dttname].resample('D').median()
            etmp3 = allbut[errname].resample('D').median()

        elif center == "mean":
            tmp3 = allbut[dttname].resample('D').mean()
            etmp3 = allbut[errname].resample('D').mean()
        else :
            print('Choose median or mean')
    return tmp3, etmp3

In [ ]:
from msnoise.api import connect, build_movstack_datelist

# connect to the database
db = connect()

# Obtain a list of dates between ``start_date`` and ``enddate``
start, end, datelist = build_movstack_datelist(db)

mov_stack = 1
center = 'mean'
# Get the dtt results for all stations, filter id=1, ZZ component, mov_stack=1 and the error associated:
dtt, edtt = get_dvv(db, 1, "ZZ", datelist, mov_stack=mov_stack, center=center)

fig = plt.figure(figsize=(12, 9))

plt.plot(dtt.index, dtt, label=center)
plt.fill_between(dtt.index, dtt-edtt,
                     dtt+edtt, zorder=-1,
                     alpha=0.5, label='error')
plt.ylabel('dv/v (%)')

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=4,
           ncol=2, borderaxespad=0.)
left, right = dtt.index[0], dtt.index[-1]
plt.title('%i Days Moving Window' % mov_stack)

plt.grid(True)
plt.gca().xaxis.set_major_formatter(DateFormatter("%Y-%m-%d %H:%M"))
fig.autofmt_xdate()

plt.show()


## Bonus: recompute only the last week before the flood - Cross-Components


In [ ]:
! msnoise config set components_to_compute_single_station=EN,EZ,NZ
! msnoise config set components_to_compute=""
! msnoise config set keep_all=Y
! msnoise reset DA
! msnoise new_jobs
! msnoise db execute "update jobs set flag='D' where jobtype='CC'"
! msnoise db execute "update jobs set flag='T' where jobtype='CC' and day between '2021-07-06' and '2021-07-15'"

In [ ]:
! msnoise info -j

then run msnoise cc compute_cc again ! This time, it'll generate H5 files containing the individual windows of CCFs

In [ ]:


# connect to the database
db = connect()

# Obtain a list of dates between ``start_date`` and ``enddate``
start, end, datelist = build_movstack_datelist(db)

# Get the list of parameters from the DB:
params = get_params(db)

# Get the time axis for plotting the CCF:
taxis = get_t_axis(db)

station = "GR.AHRW.--"
filter_id = 4
components = "EZ"

# Get the results for two station, filter id=1, ZZ component, mov_stack=1 and the results as a 2D array:
ccfs = get_results_all(db, station, station, filter_id, components, datelist)
print(ccfs.head())
# Convert to a pandas DataFrame object for convenience, and drop empty rows:
# df = pd.DataFrame(ccfs, index=pd.DatetimeIndex(datelist), columns=taxis)
df = ccfs.dropna()

# Define the 99% percentile of the data, for visualisation purposes:
clim = df.mean(axis="index").quantile(0.9999)

fig, ax = plt.subplots(figsize=(12,8))
plt.pcolormesh(df.columns, df.index.to_pydatetime(), df.values,
               vmin=-clim, vmax=clim, rasterized=True, cmap="RdBu")
plt.colorbar()
plt.title("Interferogram")
plt.xlabel("Lag Time (s)")
plt.ylim(df.index[0],df.index[-1])
# plt.xlim(df.columns[0], df.columns[-1])
plt.xlim(-10,10)
plt.subplots_adjust(left=0.15)

In [ ]:
df = df.loc["2021-07-13":]
fig, ax = plt.subplots(figsize=(12,8))
plt.pcolormesh(df.columns, df.index.to_pydatetime(), df.values,
               vmin=-clim, vmax=clim, rasterized=True, cmap="RdBu")
plt.colorbar()
plt.title("Interferogram")
plt.xlabel("Lag Time (s)")
plt.ylim(df.index[0],df.index[-1])
# plt.xlim(df.columns[0], df.columns[-1])
plt.xlim(-5,5)
plt.subplots_adjust(left=0.15)